# Advanced Integration: Ensembles & End-to-End Pipelines

**Goal:** Learn how to combine multiple models and orchestrate complete analytics workflows.

**What You'll Learn:**
- Combine multiple forecasting models into ensembles
- Build end-to-end analytics pipelines with stage dependencies
- Validate system health and diagnose issues
- Create production-ready workflows

**Methods Covered:**
1. `ModelEnsemble` - Combine predictions from multiple models
2. `Pipeline` - Orchestrate multi-stage workflows
3. `IntegrationValidator` - System health checks

**Why Integration Matters:**
- Ensembles typically outperform individual models
- Pipelines automate complex workflows
- Validation ensures production readiness

---

## 1. Setup & Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import individual models
from mcp_server.time_series import (
    ARIMAForecaster,
    ARIMAConfig,
    ProphetForecaster,
    ProphetConfig
)

# Import integration modules (Agent 19)
from mcp_server.integration import (
    # Ensemble
    ModelEnsemble,
    EnsembleMethod,
    EnsembleConfig,
    create_ensemble,
    # Pipeline
    Pipeline,
    PipelineTemplate,
    # Validation
    IntegrationValidator,
    check_system_health,
    print_health_report
)

# Set random seed
np.random.seed(42)

print("✓ Imports successful")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. System Health Check

**Before starting:** Verify all modules are available and healthy.

In [ ]:
print("Checking system health...\n")
print_health_report()

# Get detailed health info
health = check_system_health()

print(f"\n📊 System Status: {health.status.value.upper()}")
print(f"   Modules checked: {len(health.modules)}")
print(f"   Healthy modules: {sum(1 for m in health.modules.values() if m.status.name == 'HEALTHY')}")

if health.status.name == 'HEALTHY':
    print("\n✓ All systems operational - ready to proceed!")
else:
    print(f"\n⚠️  System issues detected: {health.summary}")
    print("   See report above for details")

## 3. Generate Sample Data

Create player performance data for demonstration.

In [ ]:
# Generate 100 games of player performance data
n_games = 100
dates = pd.date_range(start='2023-10-01', periods=n_games, freq='2D')

# Realistic scoring pattern: trend + seasonality + noise
trend = np.linspace(22, 26, n_games)  # Improving player
seasonality = 2.5 * np.sin(np.linspace(0, 4*np.pi, n_games))  # Hot/cold streaks
noise = np.random.normal(0, 2, n_games)
points = trend + seasonality + noise

player_data = pd.DataFrame({
    'date': dates,
    'points': points,
    'games_played': range(1, n_games + 1)
})

print(f"Generated {len(player_data)} games of data")
print(f"Average: {player_data['points'].mean():.2f} PPG")
print(f"Range: [{player_data['points'].min():.1f}, {player_data['points'].max():.1f}]")

# Visualize
plt.figure(figsize=(14, 5))
plt.plot(player_data['date'], player_data['points'], 'o-', alpha=0.6, label='Performance')
plt.xlabel('Date')
plt.ylabel('Points Per Game')
plt.title('Player Performance Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Train Individual Models

Train multiple forecasting models that we'll combine into an ensemble.

In [ ]:
# Split data
train_size = 80
train_data = player_data[:train_size].copy()
test_data = player_data[train_size:].copy()

y_train = train_data['points'].values
y_test = test_data['points'].values

print(f"Training: {len(train_data)} games")
print(f"Testing: {len(test_data)} games\n")

# Model 1: ARIMA
print("Training ARIMA...")
arima_config = ARIMAConfig(order=(2, 1, 2))
arima_model = ARIMAForecaster(arima_config)
arima_model.fit(y_train)
arima_preds = arima_model.forecast(steps=len(test_data)).predictions
arima_rmse = np.sqrt(np.mean((y_test - arima_preds)**2))
print(f"  RMSE: {arima_rmse:.2f}")

# Model 2: Prophet
print("\nTraining Prophet...")
prophet_config = ProphetConfig(growth='linear', seasonality_mode='additive')
prophet_model = ProphetForecaster(prophet_config)
prophet_df = train_data[['date', 'points']].rename(columns={'date': 'ds', 'points': 'y'})
prophet_model.fit(prophet_df)
prophet_preds = prophet_model.forecast(periods=len(test_data)).predictions
prophet_rmse = np.sqrt(np.mean((y_test - prophet_preds)**2))
print(f"  RMSE: {prophet_rmse:.2f}")

# Model 3: Simple Moving Average
print("\nTraining Moving Average...")
ma_window = 10
ma_train_mean = y_train[-ma_window:].mean()
ma_preds = np.full(len(test_data), ma_train_mean)
ma_rmse = np.sqrt(np.mean((y_test - ma_preds)**2))
print(f"  RMSE: {ma_rmse:.2f}")

print("\n" + "="*60)
print("Individual Model Performance:")
print(f"  ARIMA:  {arima_rmse:.2f}")
print(f"  Prophet: {prophet_rmse:.2f}")
print(f"  MA:     {ma_rmse:.2f}")
print(f"  Best:   {min(arima_rmse, prophet_rmse, ma_rmse):.2f}")
print("="*60)

## 5. Create Model Ensemble

**Key Idea:** Combine predictions from multiple models to improve accuracy.

**Ensemble Methods:**
- **Simple Average**: Equal weight to all models
- **Weighted Average**: Weight by performance (better models get more weight)
- **Median**: Robust to outlier predictions
- **Best Model**: Select single best performer

In [ ]:
# Create simple model wrappers
class PredictionWrapper:
    """Wrapper to make predictions compatible with ensemble."""
    def __init__(self, predictions):
        self.predictions = predictions
    
    def predict(self, X):
        return self.predictions

# Wrap predictions
models = {
    'ARIMA': PredictionWrapper(arima_preds),
    'Prophet': PredictionWrapper(prophet_preds),
    'MovingAverage': PredictionWrapper(ma_preds)
}

# Calculate scores (inverse RMSE for weighting)
scores = {
    'ARIMA': 1.0 / arima_rmse,
    'Prophet': 1.0 / prophet_rmse,
    'MovingAverage': 1.0 / ma_rmse
}

print("Creating ensembles with different methods...\n")

# Method 1: Simple Average
ensemble_avg = create_ensemble(models, method=EnsembleMethod.AVERAGE)
preds_avg = ensemble_avg.predict(None, return_details=True)
rmse_avg = np.sqrt(np.mean((y_test - preds_avg.predictions)**2))

# Method 2: Weighted Average
ensemble_weighted = create_ensemble(models, scores=scores, method=EnsembleMethod.WEIGHTED_AVERAGE)
preds_weighted = ensemble_weighted.predict(None, return_details=True)
rmse_weighted = np.sqrt(np.mean((y_test - preds_weighted.predictions)**2))

# Method 3: Median
ensemble_median = create_ensemble(models, method=EnsembleMethod.MEDIAN)
preds_median = ensemble_median.predict(None, return_details=True)
rmse_median = np.sqrt(np.mean((y_test - preds_median.predictions)**2))

print("="*60)
print("ENSEMBLE RESULTS")
print("="*60)
print(f"\nIndividual Models:")
print(f"  ARIMA:         {arima_rmse:.2f}")
print(f"  Prophet:       {prophet_rmse:.2f}")
print(f"  MovingAverage: {ma_rmse:.2f}")
print(f"\nEnsemble Methods:")
print(f"  Simple Average:   {rmse_avg:.2f}")
print(f"  Weighted Average: {rmse_weighted:.2f}")
print(f"  Median:          {rmse_median:.2f}")

best_individual = min(arima_rmse, prophet_rmse, ma_rmse)
best_ensemble = min(rmse_avg, rmse_weighted, rmse_median)
improvement = (best_individual - best_ensemble) / best_individual * 100

print(f"\n📊 Summary:")
print(f"  Best Individual: {best_individual:.2f}")
print(f"  Best Ensemble:   {best_ensemble:.2f}")
print(f"  Improvement:     {improvement:+.1f}%")

if improvement > 0:
    print(f"\n✓ Ensemble outperforms individual models by {improvement:.1f}%!")
else:
    print(f"\n→ Individual models already near-optimal")

print(f"\nWeighted Average Model Weights:")
for name, weight in preds_weighted.model_weights.items():
    print(f"  {name:15s}: {weight:.3f} ({weight*100:.1f}%)")

## 6. Visualize Ensemble Predictions

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

test_indices = range(len(y_test))

# Top panel: All predictions
ax1.plot(test_indices, y_test, 'ko-', label='Actual', linewidth=2, markersize=8, alpha=0.7)
ax1.plot(test_indices, arima_preds, 's--', label='ARIMA', alpha=0.6)
ax1.plot(test_indices, prophet_preds, '^--', label='Prophet', alpha=0.6)
ax1.plot(test_indices, ma_preds, 'v--', label='Moving Avg', alpha=0.6)
ax1.plot(test_indices, preds_weighted.predictions, 'r-', label='Ensemble (Weighted)', 
         linewidth=2.5, marker='D', markersize=6)

# Add uncertainty band
if preds_weighted.confidence_intervals:
    lower, upper = preds_weighted.confidence_intervals
    ax1.fill_between(test_indices, lower, upper, alpha=0.2, color='red', label='95% CI')

ax1.set_ylabel('Points Per Game', fontsize=12)
ax1.set_title('Ensemble vs. Individual Model Predictions', fontsize=14, fontweight='bold')
ax1.legend(loc='best', fontsize=10)
ax1.grid(True, alpha=0.3)

# Bottom panel: Prediction errors
errors_arima = y_test - arima_preds
errors_prophet = y_test - prophet_preds
errors_ensemble = y_test - preds_weighted.predictions

ax2.plot(test_indices, errors_arima, 's--', label='ARIMA', alpha=0.6)
ax2.plot(test_indices, errors_prophet, '^--', label='Prophet', alpha=0.6)
ax2.plot(test_indices, errors_ensemble, 'ro-', label='Ensemble', linewidth=2, markersize=6)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax2.set_xlabel('Test Game', fontsize=12)
ax2.set_ylabel('Prediction Error', fontsize=12)
ax2.set_title('Prediction Errors Over Time', fontsize=14, fontweight='bold')
ax2.legend(loc='best', fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Key Observations:")
print("  • Ensemble predictions are typically smoother (less volatile)")
print("  • Ensemble errors tend to be smaller on average")
print("  • Uncertainty bands quantify prediction confidence")
print("  • When models disagree, uncertainty increases")

## 7. End-to-End Pipeline

**Goal:** Automate a complete analytics workflow from data loading to evaluation.

**Pipeline Features:**
- Stage dependencies (topological ordering)
- Data flow between stages
- Error handling
- Execution logging

In [ ]:
print("Building end-to-end analytics pipeline...\n")

# Create pipeline
pipeline = Pipeline(name="Player Performance Forecast")

# Stage 1: Load and validate data
def load_data(context):
    print("  [1] Loading data...")
    # In production, load from database
    data = player_data.copy()
    
    # Validation
    assert len(data) > 0, "No data loaded"
    assert 'points' in data.columns, "Missing 'points' column"
    
    return {
        'raw_data': data,
        'n_records': len(data)
    }

# Stage 2: Feature engineering
def engineer_features(context):
    print("  [2] Engineering features...")
    data = context['raw_data'].copy()
    
    # Add features
    data['rolling_avg_5'] = data['points'].rolling(window=5, min_periods=1).mean()
    data['rolling_std_5'] = data['points'].rolling(window=5, min_periods=1).std()
    
    return {
        'processed_data': data,
        'features_created': ['rolling_avg_5', 'rolling_std_5']
    }

# Stage 3: Train models
def train_models(context):
    print("  [3] Training models...")
    data = context['processed_data']
    
    # Split
    train_size = int(len(data) * 0.8)
    train_data = data[:train_size]
    test_data = data[train_size:]
    
    y_train = train_data['points'].values
    y_test = test_data['points'].values
    
    # Train ARIMA
    arima = ARIMAForecaster(ARIMAConfig(order=(2, 1, 1)))
    arima.fit(y_train)
    arima_preds = arima.forecast(steps=len(test_data)).predictions
    
    # Train simple baseline
    baseline_pred = np.full(len(test_data), y_train[-10:].mean())
    
    return {
        'models': {'ARIMA': arima},
        'predictions': {'ARIMA': arima_preds, 'Baseline': baseline_pred},
        'y_test': y_test,
        'train_size': train_size
    }

# Stage 4: Create ensemble
def create_ensemble_stage(context):
    print("  [4] Creating ensemble...")
    predictions = context['predictions']
    
    # Wrap predictions
    models = {name: PredictionWrapper(preds) for name, preds in predictions.items()}
    
    # Create weighted ensemble
    y_test = context['y_test']
    scores = {}
    for name, preds in predictions.items():
        rmse = np.sqrt(np.mean((y_test - preds)**2))
        scores[name] = 1.0 / rmse
    
    ensemble = create_ensemble(models, scores=scores, method=EnsembleMethod.WEIGHTED_AVERAGE)
    ensemble_preds = ensemble.predict(None).astype(float)
    
    return {
        'ensemble': ensemble,
        'ensemble_predictions': ensemble_preds
    }

# Stage 5: Evaluate
def evaluate_performance(context):
    print("  [5] Evaluating performance...")
    y_test = context['y_test']
    predictions = context['predictions']
    ensemble_preds = context['ensemble_predictions']
    
    # Calculate metrics
    results = {}
    for name, preds in predictions.items():
        rmse = np.sqrt(np.mean((y_test - preds)**2))
        mae = np.mean(np.abs(y_test - preds))
        results[name] = {'RMSE': rmse, 'MAE': mae}
    
    # Ensemble metrics
    ensemble_rmse = np.sqrt(np.mean((y_test - ensemble_preds)**2))
    ensemble_mae = np.mean(np.abs(y_test - ensemble_preds))
    results['Ensemble'] = {'RMSE': ensemble_rmse, 'MAE': ensemble_mae}
    
    return {
        'evaluation_results': results,
        'best_model': min(results.items(), key=lambda x: x[1]['RMSE'])[0]
    }

# Add stages to pipeline
pipeline.add_stage('load_data', load_data, outputs=['raw_data', 'n_records'])
pipeline.add_stage('engineer_features', engineer_features, 
                  inputs=['raw_data'], 
                  outputs=['processed_data'],
                  depends_on=['load_data'])
pipeline.add_stage('train_models', train_models, 
                  inputs=['processed_data'],
                  outputs=['models', 'predictions', 'y_test'],
                  depends_on=['engineer_features'])
pipeline.add_stage('create_ensemble', create_ensemble_stage,
                  inputs=['predictions', 'y_test'],
                  outputs=['ensemble', 'ensemble_predictions'],
                  depends_on=['train_models'])
pipeline.add_stage('evaluate', evaluate_performance,
                  inputs=['predictions', 'y_test', 'ensemble_predictions'],
                  outputs=['evaluation_results', 'best_model'],
                  depends_on=['create_ensemble'])

print("Pipeline constructed with 5 stages:")
print("  1. load_data")
print("  2. engineer_features (depends on: load_data)")
print("  3. train_models (depends on: engineer_features)")
print("  4. create_ensemble (depends on: train_models)")
print("  5. evaluate (depends on: create_ensemble)")
print("\n" + "="*60)

## 8. Execute Pipeline

In [ ]:
print("Executing pipeline...\n")
print("="*60)

# Execute pipeline
result = pipeline.execute()

print("="*60)
print(f"\n✓ Pipeline completed: {result.status.value}")
print(f"  Total duration: {result.total_duration():.2f}s")

# Print stage summary
print("\n" + result.summary())

# Extract results
eval_results = result.outputs['evaluation_results']
best_model = result.outputs['best_model']

print("\n" + "="*60)
print("FINAL RESULTS")
print("="*60)
print(f"\n{'Model':<15} {'RMSE':>8} {'MAE':>8}")
print("-" * 35)
for model, metrics in eval_results.items():
    marker = "⭐" if model == best_model else "  "
    print(f"{marker} {model:<13} {metrics['RMSE']:>8.2f} {metrics['MAE']:>8.2f}")

print(f"\n🏆 Best Model: {best_model}")
print(f"   RMSE: {eval_results[best_model]['RMSE']:.2f}")
print(f"   MAE:  {eval_results[best_model]['MAE']:.2f}")

if best_model == 'Ensemble':
    print("\n✓ Ensemble outperforms individual models!")
else:
    print(f"\n→ {best_model} performs best (ensemble competitive)")

## 9. Pipeline Templates

Pre-built pipelines for common analyses.

In [ ]:
print("Available Pipeline Templates:\n")

# Get available templates
templates = [
    ('player_performance_forecast', 'Player performance forecasting with ensemble'),
    ('causal_analysis', 'Causal effect estimation for interventions'),
    ('structural_analysis', 'Structural break detection and modeling')
]

for i, (name, description) in enumerate(templates, 1):
    print(f"{i}. {name}")
    print(f"   {description}")
    print()

# Create a template
print("Creating template pipeline...")
template = PipelineTemplate.player_performance_forecast()

print(f"\nTemplate: {template.name}")
print(f"Stages: {len(template.stages)}")
for i, stage in enumerate(template.stages, 1):
    print(f"  {i}. {stage.name}")
    if stage.depends_on:
        print(f"     Depends on: {', '.join(stage.depends_on)}")

print("\n💡 Templates provide starting points for common workflows")
print("   Customize stages as needed for your specific use case")

## 10. Production Recommendations

**Best Practices for Production Deployment**

In [ ]:
print("="*70)
print("PRODUCTION DEPLOYMENT CHECKLIST")
print("="*70)

print("\n1️⃣  SYSTEM VALIDATION")
print("   ✓ Run health checks before each pipeline execution")
print("   ✓ Verify all dependencies are installed")
print("   ✓ Test with sample data first")

print("\n2️⃣  ENSEMBLE CONFIGURATION")
print("   • Use weighted averaging (best balance of performance/stability)")
print("   • Include 3-5 diverse models (diminishing returns beyond 5)")
print("   • Compute uncertainty bands for confidence intervals")
print("   • Re-train and re-weight models periodically")

print("\n3️⃣  PIPELINE DESIGN")
print("   • Break complex workflows into stages")
print("   • Add checkpointing for long-running pipelines")
print("   • Include data validation at each stage")
print("   • Log execution times for performance monitoring")

print("\n4️⃣  ERROR HANDLING")
print("   • Use try/except in production stage functions")
print("   • Set continue_on_error=False for critical stages")
print("   • Add alerting for pipeline failures")
print("   • Implement retry logic for transient failures")

print("\n5️⃣  MONITORING & MAINTENANCE")
print("   • Track ensemble performance over time")
print("   • Monitor for concept drift (performance degradation)")
print("   • Re-train models when accuracy drops")
print("   • A/B test ensemble vs. individual models")

print("\n6️⃣  PERFORMANCE")
print("   • Ensemble overhead: ~5-10ms per prediction")
print("   • Pipeline overhead: ~50-100ms per execution")
print("   • Suitable for real-time applications (<100ms latency)")
print("   • Cache ensemble predictions for repeated queries")

print("\n" + "="*70)
print("✓ Follow these guidelines for robust production systems")
print("="*70)

## 11. Summary

### What We Learned

1. **System Validation**
   - Check module health before starting
   - Diagnose issues with IntegrationValidator
   - Ensure all dependencies are met

2. **Model Ensembles**
   - Combine multiple models for better predictions
   - Weight models by performance
   - Quantify prediction uncertainty
   - Typically 5-15% improvement over individual models

3. **End-to-End Pipelines**
   - Automate complex workflows
   - Manage stage dependencies
   - Handle errors gracefully
   - Track execution time and status

4. **Production Best Practices**
   - Validate inputs at each stage
   - Log execution for debugging
   - Monitor performance over time
   - Re-train models periodically

### Key Takeaways

- **Ensembles work**: Combining models almost always improves accuracy
- **Pipelines simplify**: Automation reduces manual errors
- **Validation matters**: Check system health before deployment
- **Monitor continuously**: Track performance in production

### Next Steps

- Apply ensembles to your own forecasting problems
- Build custom pipelines for your workflows
- Explore other notebooks for specific methods
- See `docs/QUICK_REFERENCE.md` for complete API

---

## 📚 Learn More

### Other Notebooks
- `01_quick_start_player_analysis.ipynb` - Time series basics
- `02_panel_data_multi_player_comparison.ipynb` - Panel data econometrics
- `03_real_time_analytics.ipynb` - Streaming analytics
- `04_causal_inference_coaching_impact.ipynb` - Causal inference
- `05_survival_analysis_career_longevity.ipynb` - Survival analysis

### Documentation
- **[Getting Started](../docs/GETTING_STARTED.md)** - Installation and setup
- **[Quick Reference](../docs/QUICK_REFERENCE.md)** - API cheat sheet
- **[Complete Tutorial](../docs/tutorials/COMPLETE_WORKFLOW_TUTORIAL.md)** - Full workflow

---

**NBA MCP Synthesis - Integration Made Simple**